# 

In [1]:
# Import libraries
import requests
import re # regular expressions
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import unidecode
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from time import sleep

In [2]:
# local variables
subCategorListTitle  = "j-menu-item"
CHROME_DRIVER_PATH = r'C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe'
URL = 'https://www.wildberries.ru/catalog/muzhchinam/odezhda/bryuki-i-shorty'


driver = webdriver.Chrome(CHROME_DRIVER_PATH)
driver.get(URL)
driver.maximize_window()

content = driver.page_source
soup = BeautifulSoup(content)

prices = []
brands = []
categories = []
buyCountList = []
ratings = []
data = {
        "Categories" : categories,
        "Brand" : brands,
        "Price" : prices,
        "BuyCount" : buyCountList,
        "Ratings" : ratings
}

df = pd.DataFrame(data)


writer = pd.ExcelWriter('data.xlsx', engine='openpyxl')


In [3]:
#To wait till page loads
def waitTillPageLoads(classNameLoaded, URL_name):
    driver.get(URL_name)
    delay = 10
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, classNameLoaded)))
        print("Page is ready!")
    except TimeoutException:
        print("Loading took too much time!")
        
waitTillPageLoads('lower-price', driver.current_url)


Page is ready!


In [4]:
def searchAllItemsOfCurrentPage(prices, brands, categories, buyCountList, ratings):
    class_title = 'j-card-item'
    sizes_available = []
    content = driver.page_source
    soup = BeautifulSoup(content)
    for block in soup.findAll('div', attrs={'class':class_title}):
        class_title = 'j-card-item'
        
        # Get price of clothes
        priceString = "0"
        priceBlock = block.find('ins', attrs={'class': 'lower-price'})
        if(priceBlock != None):
            priceString = block.find('ins', attrs={'class': 'lower-price'}).text
        elif (block.find('span', attrs={'class': 'lower-price'}) == None):
            priceString = "0"
        else:
            priceString = block.find('span', attrs={'class': 'lower-price'}).text
        price = unidecode.unidecode(priceString.strip())
        prices.append(price)
        
        # Get brand of clothes
        brandString = block.find('strong', attrs={'class': 'brand-name'}).text
        brand = unidecode.unidecode(brandString.strip())
        brands.append(brand)

        # Get category of clothes
        categoryString = block.find('span', attrs={'class': 'goods-name'}).text
        category = unidecode.unidecode(categoryString.strip())
        categories.append(category)

        #Get count of bought times
        buyCount = block.find('span', attrs={'class': 'product-card__count'}).text
        buyCount = unidecode.unidecode(buyCount.strip())
        buyCountList.append(buyCount)
        
        #Get rating of clothes
        rating =  block.find('span', attrs={'class': 'product-card__rating'}).text
        rating = unidecode.unidecode(rating.strip())
        ratings.append(rating)

In [5]:
def addDataToDF(list, df):
    count = 0
    df2 = pd.DataFrame(data=list).T
    df2.columns = ["Categories", "Brand", "Price", "BuyCount", "Ratings"]
    df = pd.concat([df, df2])
    df.to_excel(writer,index=False, sheet_name='All data')
    writer.save()

In [6]:
prices = []
brands = []
categories = []
buyCountList = []
ratings = []

#Search for shoes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    shoesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--629")
    ActionChains(driver).click(burgerButton).pause(1).click(shoesButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories, buyCountList, ratings)
    count+=1
    
    if count == 5:
        break

        
        
        
        
#Search for women clothes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    forWomenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--306")
    ActionChains(driver).click(burgerButton).pause(1).click(forWomenButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories, buyCountList, ratings)
    count+=1
    
    if count == 28:
        break
 

#Search for babies clothes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    forBabiesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--115")
    ActionChains(driver).click(burgerButton).pause(1).click(forBabiesButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories, buyCountList, ratings)
    count+=1
    
    if count == 2:
        break
 
#Search for men clothes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    forMenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--566")
    ActionChains(driver).click(burgerButton).pause(1).click(forMenButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories, buyCountList, ratings)
    count+=1
    
    if count == 26:
        break

#Search for accessories
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    accessoriesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--1")
    ActionChains(driver).click(burgerButton).pause(1).click(accessoriesButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories, buyCountList, ratings)
    count+=1
    
    if count == 18:
        break
res = []        
res.append(categories)
res.append(brands)
res.append(prices)
res.append(buyCountList)
res.append(ratings)
addDataToDF(res, df)



Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is ready!
Page is re

In [7]:
#Preprocessing and writing data to excel.

df = pd.read_excel('data.xlsx', engine='openpyxl')
df = df.reset_index()  # make sure indexes pair with number of rows

dict_categories = {}
dict_brand_occurences = {}
dict_brand_price = {}
for index, row in df.iterrows():
    
    #count all word occurrences in categories column
    category = row['Categories']
    category = category.replace('-', '')
    category = category.replace(',', '')
    category_list = category.split(" ")
    for word in category_list:
        dict_categories[word] = dict_categories.get(word, 0) + 1
    
    
    
    #count each company names in column brand
    #sum of price for each company
    brand = row['Brand']
    price = row['Price']
    count = row['BuyCount']
    price =  int(price.replace(" ", "").replace("R","")) * int(count.replace(" ", "").replace("R","")) 
    
    dict_brand_occurences[brand] = dict_brand_occurences.get(brand, 0) + 1
    dict_brand_price[brand] = dict_brand_price.get(brand, 0) + price
    

    
    
#sorting dictionary    
dict_categories = dict(sorted(dict_categories.items(), key=lambda item: item[1], reverse=True))

#writiting data to excel
df_categories_occurrences = pd.DataFrame(data=dict_categories, index=[1])
df_categories_occurrences = (df_categories_occurrences.T)
df_categories_occurrences.reset_index(inplace=True)
df_categories_occurrences = df_categories_occurrences.rename({'index': 'Categories', 1: 'Occurrences'}, axis=1)
df_categories_occurrences.to_excel(writer, sheet_name='Categories Occurrences', index=False)
writer.save()

#sorting dictionaries
dict_brand_occurences = dict(sorted(dict_brand_occurences.items(), key=lambda item: item[1], reverse=True))
dict_brands_price = dict(sorted(dict_brand_price.items(), key=lambda item: item[1], reverse=True))


df_brand_occurrences = pd.DataFrame(data=dict_brand_occurences, index = [0])
df_brand_occurrences = (df_brand_occurrences.T)


df_brands_price = pd.DataFrame(data=dict_brand_price, index = [0])
df_brands_price = (df_brands_price.T)

df_brand_occurrences.reset_index(inplace=True)
df_brand_occurrences = df_brand_occurrences.rename({'index': 'Brand', 0: 'Occurrences'}, axis=1)

df_brands_price.reset_index(inplace=True)
df_brands_price = df_brands_price.rename({'index': 'Brand', 0: 'Price'}, axis=1)

#merging two dicionaries
merged_inner = pd.merge(left=df_brand_occurrences, right=df_brands_price, left_on="Brand", right_on="Brand")

#Writing data to excel
merged_inner.to_excel(writer, sheet_name='Brand Price', index=False)
writer.save()
    